In [47]:
# 情感词抽取
import jieba.posseg as pseg
import jieba
import pandas as pd
# 哈工大停用词、四川大学机器智能实验室停用词、百度停用词、中文停用词表
# https://github.com/goto456/stopwords

aspect_words=pd.read_table('CAT法抽取到的方面词.tsv',encoding='utf-8',header=None)[0].tolist()
# 抽取<方面,情感>词组,并写入文件
stopwords=[line.strip() for line in open('D:\python_common\基本语料库\停用词表完全版.txt',encoding='utf-8').readlines()]
def find_pattern(sentence_cut_,word,index,checked):
    # sentence_cut_:[[word,flag],[word,flag]...]
    # word:当前词
    # index:当前词的索引
    # checked:已经被检查过的词的索引
    sentence_cut=sentence_cut_.copy()
    # 形容词+属性词
    if index > 0 and sentence_cut[index-1][1] in ['a','v'] and index-1 not in checked :
        checked.append(index-1)
        return  [sentence_cut[index-1][0],word],checked
    # 属性词+形容词
    elif index+1 < len(sentence_cut) and sentence_cut[index+1][1] in ['a','x','v'] and index+1 not in checked :
        checked.append(index+1)
        return  [word,sentence_cut[index+1][0]],checked
    # 属性词+形容词+形容词
    elif index+2 < len(sentence_cut) and sentence_cut[index+1][1] in ['a','x','v'] and sentence_cut[index+2][1] in ['a','x','v'] and index+2 not in checked:
        checked.append(index+2)
        return  [word,sentence_cut[index+1][0],sentence_cut[index+2][0]],checked
    # 形容词+形容词+属性词
    elif index+2 < len(sentence_cut) and sentence_cut[index+1][1] in ['a','x','v'] and sentence_cut[index+2][1] in ['a','x','v'] and index+2 not in checked:
        checked.append(index+2)
        return  [word,sentence_cut[index+1][0],sentence_cut[index+2][0]],checked
    # 属性词+副词+形容词
    elif index+2 < len(sentence_cut) and sentence_cut[index+1][1] in ['d','x','zg'] and sentence_cut[index+2][1] in ['a','v','x'] and index+2 not in checked :
        checked.append(index+2)
        return  [word,sentence_cut[index+1][0],sentence_cut[index+2][0]],checked
    # 副词+形容词+属性词
    elif 2 < index < len(sentence_cut)-1 and sentence_cut[index - 2][1] in ['d', 'x'] and sentence_cut[index - 1][1] in ['a', 'v', 'x'] and index-2 not in checked:
        checked.append(index-2)
        return  [sentence_cut[index-2][0],sentence_cut[index-1][0],word],checked

    # 返回格式:[[],[]...],checked
    return [],checked

def check(sentence):
    # 切词
    sentence_cut_=pseg.lcut(sentence.strip())
    sentence_cut=[[each.word,each.flag] for each in sentence_cut_ if each.word not in stopwords]
    # S为抽取结果，checked为已经被检查过的词的索引
    S=[];checked=[]
    for i in range(len(sentence_cut)):
        if sentence_cut[i][0] in aspect_words:
            # 找到关键名词
            ss,checked=find_pattern(sentence_cut,sentence_cut[i][0],i,checked)
            # 匹配到模式
            ss=list(set(ss))
            # 去重
            ss=[each for each in ss if each !=' ']
            # 找到的关键词中没有空格，最少两个
            if  len(ss)>1:
                S.append('#'.join(ss))
    # 一个句子所有的抽取结果，大于等于1次
    if len(S)>=1:
        return '@'.join(S)
    # 如果抽取结果为空
    else:
        S=[]

        [S.append(each) for each in jieba.lcut(sentence.strip()) if each not in S
         and each not in stopwords and each not in [' ','\n','\t','\r','\u3000','\xa0']
         and '一' not in each]
        return '#'.join(S)


for path in ['train.csv','test.csv','dev.csv']:
    print(path,'start')
    data=pd.read_csv('预处理结果_'+path,encoding='utf-8')
    datas=[]
    for i in range(0,len(data['review'])):
        datas.append(check(data['review'][i]))
        if i%10==0:
            print('\r',str(100*i/len(data['review']))[0:6]+'%',end='')
        if i == len(data['review'])-1:
            print('\r', '100.00%', end='')

    data['keywords']=datas
    data.to_csv('关键词抽取结果_'+path,encoding='utf-8',index=False)
    print('\n',path,'done,save to 关键词抽取结果_'+path,'total:',len(data['review']),'rows')


train.csv start
 99.997%
 train.csv done
test.csv start
 99.979%
 test.csv done
dev.csv start
 99.979%
 dev.csv done
